In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sc

### **Merge ERA5 files:**

In [ ]:
#use dask in xarray to merge
ds = xr.open_mfdataset('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//ERA5_2//*.nc')
#save it
ds.to_netcdf('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//ERA5_2//ERA5.hourly.1981-2021.nc')
#open data
era5 = xr.open_dataset('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//ERA5_2//ERA5.hourly.1981-2021.nc')

### **Merge HAMMOZ model files:**

In [ ]:
ds2 = xr.open_mfdataset('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//Hammoz_historical//*.nc')
ds2.to_netcdf('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//Hammoz_historical//ham_hist.nc')
ds3 = xr.open_mfdataset('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//Hammoz_ssp//*.nc')
ds3.to_netcdf('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//Hammoz_ssp//ham_ssp.nc')

### **Interpolate ERA5 and HAMMOZ files:**

In [ ]:
#interpolate era5 and save it
era5_int = era5.t2m.interp(latitude=56, longitude=14)
era5_int.to_netcdf('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//ERA5_2//ERA5.1981-2014_int.nc')
#interpolate HAMMOZ historical and save it
ham_hist_int = ham_hist.tas.interp(lat=56, lon=14)
ham_hist_int.to_netcdf('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//Hammoz_historical//ham_hist_interpo.nc')
#interpolate HAMMOZ ssp and save it
ham_ssp_int = ham_ssp.tas.interp(lat=56, lon=14)
ham_ssp_int.to_netcdf('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//Hammoz_ssp//ham_ssp_interpo.nc')

#open datasets
eraint = xr.open_dataset('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//ERA5_2//ERA5.1981-2014_int.nc')
model = xr.open_dataset('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//Hammoz_historical//ham_hist_interpo.nc')

### **Comparison between 4- and 7 month periods (ERA5 data), Chilling Hours Model:**

In [ ]:
#select years
#choosing the winter months
#converting from kelvin
time_slice = slice('1981-01-01T00:00:00.000000000','2014-12-31T23:30:00.000000000')
erasel = eraint.sel(time=time_slice)
era4month = erasel.t2m.sel(time=erasel.time.dt.month.isin([1, 2, 3, 12])) - 273.15
era7month = erasel.t2m.sel(time=erasel.time.dt.month.isin([1, 2, 3, 4, 10, 11, 12])) - 273.15

#Chilling Hours Model calculation, 4 month period:
temp=era4month
chill_hours = xr.where((temp>= 0) & (temp <= 7.2), 1 , np.nan)
chill_hours_year = chill_hours.groupby('time.year').sum(dim='time')

#Chilling Hours Model calculation, 7 month period:
temp=era7month
chill_hours = xr.where((temp>= 0) & (temp <= 7.2), 1 , np.nan)
chill_hours_year2 = chill_hours.groupby('time.year').sum(dim='time')

#plot them together
fig, ax = plt.subplots(figsize=(12, 7))
chill_hours_year.plot(ax = ax, x='year',color = 'cornflowerblue', label='ERA5 reanalysis data, 4 months')
chill_hours_year2.plot(ax = ax, x='year', color = 'midnightblue', label='ERA5 reanalysis data, 7 months')
plt.title("Winter chill hours - yearly count")
ax.set_ylabel('Hours')
ax.set_xlabel('Year')
ax.legend(loc='upper right')
plt.show()

### **Comparison between 4- and 7 month periods (ERA5 data), Utah Model:**

In [ ]:
#Utah Model calculation, 4 month period:
#conditions
temp=era4month
temp1=xr.where((temp>= 1.5) & (temp <= 2.4), 0.5 , np.nan)
temp2=xr.where((temp>= 2.5) & (temp <= 9.1), 1, np.nan)
temp3=xr.where((temp>= 9.2) & (temp <= 12.4), 0.5 , np.nan)
temp4=xr.where((temp>= 16) & (temp <= 18), (-0.5) , np.nan)
temp5=xr.where((temp>= 18), -1 , np.nan) 

yearsum1 = temp1.groupby('time.year').sum(dim='time')
yearsum2 = temp2.groupby('time.year').sum(dim='time')
yearsum3 = temp3.groupby('time.year').sum(dim='time')
yearsum4 = temp4.groupby('time.year').sum(dim='time')
yearsum5 = temp5.groupby('time.year').sum(dim='time')

yearsum = yearsum1 + yearsum2 + yearsum3 + yearsum4 + yearsum5

#Utah Model calculation, 7 month period:
#conditions
temp=era7month
temp1=xr.where((temp>= 1.5) & (temp <= 2.4), 0.5 , np.nan)
temp2=xr.where((temp>= 2.5) & (temp <= 9.1), 1, np.nan)
temp3=xr.where((temp>= 9.2) & (temp <= 12.4), 0.5 , np.nan)
temp4=xr.where((temp>= 16) & (temp <= 18), (-0.5) , np.nan)
temp5=xr.where((temp>= 18), -1 , np.nan)

yearsum1 = temp1.groupby('time.year').sum(dim='time')
yearsum2 = temp2.groupby('time.year').sum(dim='time')
yearsum3 = temp3.groupby('time.year').sum(dim='time')
yearsum4 = temp4.groupby('time.year').sum(dim='time')
yearsum5 = temp5.groupby('time.year').sum(dim='time')

yearsum2 = yearsum1 + yearsum2 + yearsum3 + yearsum4 + yearsum5

#plot them together
fig, ax = plt.subplots(figsize=(12, 7))
yearsum.plot(ax = ax, x='year',color = 'cornflowerblue', label='ERA5 reanalysis data, 4 months')
yearsum2.plot(ax = ax, x='year', color = 'midnightblue', label='ERA5 reanalysis data, 7 months')
plt.title("Utah chill hour units - yearly count")
ax.set_ylabel('Units')
ax.set_xlabel('Year')
ax.legend(loc='upper right')
plt.show()

### **Calculating mean from ERA5 data as a baseline for chilling requirements:**

In [ ]:
print('mean for chill units 4 months:', yearsum.mean())
print('mean for chill units 7 months:', yearsum2.mean())
print('mean for chill hours 4 months:', chill_hours_year.mean())
print('mean for chill hours 7 months:', chill_hours_year2.mean())

### **Plotting ERA5 data and mean:**

In [ ]:
#Chilling Hours Model
fig, ax = plt.subplots(figsize=(12, 7))
chill_hours_year2.plot(x='year', color = 'midnightblue', label='ERA5 reanalysis data')
plt.title("Winter chill hours - yearly count")
ax.set_ylabel('Hours')
ax.set_xlabel('Year')
ax.axhline(y=chill_hours_year2.mean('year'), color='peachpuff', label='Mean = 2683', linewidth=3)
ax.legend(loc='upper right')
plt.show()

#Utah Model
fig, ax = plt.subplots(figsize=(12, 7))
yearsum2.plot(x='year', color = 'midnightblue', label='ERA5 reanalysis data')
plt.title("Winter chill units - yearly count")
ax.set_ylabel('Units')
ax.set_xlabel('Year')
ax.axhline(y=yearsum2.mean('year'), color='peachpuff', label='Mean = 2563', linewidth=3)
ax.legend(loc='upper right')
plt.show()

### **Bias calculation:**

In [ ]:
#select years
time_slice = slice('1981-01-01T00:00:00.000000000','2014-12-31T23:30:00.000000000')
modelsel = model.sel(time=time_slice)
erasel = eraint.sel(time=time_slice)

#dropping everything except time
#choosing the winter months
#converting from kelvin
model_ham = modelsel.tas.drop_vars(['lat', 'lon','height']).sel(time=modelsel.time.dt.month.isin([1, 2, 3, 4, 10, 11, 12])) - 273.15
era = erasel.t2m.drop_vars(['latitude', 'longitude']).sel(time=erasel.time.dt.month.isin([1, 2, 3, 4, 10, 11, 12])) - 273.15

model_monthly = model_ham.groupby('time.month').mean(dim='time', skipna=True)
era_monthly = era.groupby('time.month').mean(dim='time', skipna=True)

#monthly bias plot
bias_monthly = era_monthly-model_monthly
plt.figure(figsize=(8, 4))
bias_monthly.plot()
plt.ylabel('Bias(°C)')
plt.title('')

In [ ]:
#save it
bias_monthly.to_netcdf('C://Users//frida//Documents//Dokument//Klimatdatatillampningar//Project//bias_monthly.nc')

### **Bias correction:**

In [ ]:
bias_set = xr.open_dataset('C://Users//frida//Documents//Dokument//Klimatdatatillampningar//Project//bias_monthly.nc')
bias = bias_set.__xarray_dataarray_variable__.values

model_ham_grouped = model_ham.groupby(model_ham.time.dt.month)
months = [1, 2, 3, 4, 10, 11, 12]
bias_index = 0
for month in months:
    model_ham_without_bias = model_ham_grouped.apply(lambda x: x - bias[bias_index])
    bias_index = bias_index + 1

### **Calculating apple chill models for historical HAMMOZ model:**

In [ ]:
temp=model_ham_without_bias

#Chilling Hours Model
chill_hours = xr.where((temp>= 0) & (temp <= 7.2), 1 , np.nan)
chill_hours_year = chill_hours.groupby('time.year').sum(dim='time')

#Utah Model
temp1=xr.where((temp>= 1.5) & (temp <= 2.4), 0.5 , np.nan)
temp2=xr.where((temp>= 2.5) & (temp <= 9.1), 1, np.nan)
temp3=xr.where((temp>= 9.2) & (temp <= 12.4), 0.5 , np.nan)
temp4=xr.where((temp>= 12.5) & (temp <= 15.9), 0 , np.nan)
temp5=xr.where((temp>= 16) & (temp <= 18), (-0.5) , np.nan)
temp6=xr.where((temp>= 18), -1 , np.nan)
hours = temp1 + temp2 + temp3 + temp4 + temp5 + temp6

yearsum1 = temp1.groupby('time.year').sum(dim='time')
yearsum2 = temp2.groupby('time.year').sum(dim='time')
yearsum3 = temp3.groupby('time.year').sum(dim='time')
yearsum4 = temp4.groupby('time.year').sum(dim='time')
yearsum5 = temp5.groupby('time.year').sum(dim='time')
yearsum6 = temp6.groupby('time.year').sum(dim='time')

yearsum3 = yearsum1 + yearsum2 + yearsum3 + yearsum4 + yearsum5 + yearsum6

### **Calculating apple chill models for HAMMOZ ssp model:**

In [ ]:
model_ssp = xr.open_dataset('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//Hammoz_ssp//ham_ssp_interpo.nc')
model_ham_ssp = model_ssp.tas.drop_vars(['lat', 'lon','height']).sel(time=model_ssp.time.dt.month.isin([1, 2, 3, 4, 10, 11, 12])) - 273.15

#bias correction of ssp model
model_ham_grouped_ssp = model_ham_ssp.groupby(model_ham_ssp.time.dt.month)
months = [1, 2, 3, 4, 10, 11, 12]
bias_index = 0
for month in months:
    model_ham_without_bias_ssp = model_ham_grouped_ssp.apply(lambda x: x - bias[bias_index])
    bias_index = bias_index + 1


temp=model_ham_without_bias_ssp

#Chilling Hours Model
chill_hours = xr.where((temp>= 0) & (temp <= 7.2), 1 , np.nan)
chill_hours_year_ssp = chill_hours.groupby('time.year').sum(dim='time')

#Utah Model
temp=model_ham_without_bias_ssp
temp1=xr.where((temp>= 1.5) & (temp <= 2.4), 0.5 , np.nan)
temp2=xr.where((temp>= 2.5) & (temp <= 9.1), 1, np.nan)
temp3=xr.where((temp>= 9.2) & (temp <= 12.4), 0.5 , np.nan)
temp4=xr.where((temp>= 12.5) & (temp <= 15.9), 0 , np.nan)
temp5=xr.where((temp>= 16) & (temp <= 18), (-0.5) , np.nan)
temp6=xr.where((temp>= 18), -1 , np.nan)
hours = temp1 + temp2 + temp3 + temp4 + temp5 + temp6

yearsum1 = temp1.groupby('time.year').sum(dim='time')
yearsum2 = temp2.groupby('time.year').sum(dim='time')
yearsum3 = temp3.groupby('time.year').sum(dim='time')
yearsum4 = temp4.groupby('time.year').sum(dim='time')
yearsum5 = temp5.groupby('time.year').sum(dim='time')
yearsum6 = temp6.groupby('time.year').sum(dim='time')

yearsum_ssp = yearsum1 + yearsum2 + yearsum3 + yearsum4 + yearsum5 + yearsum6

### **Plotting everything together:**

In [ ]:
#deviding the future into two periods:
time_slice1 = slice('2015-01-01T00:00:00.000000000','2035-12-31T23:30:00.000000000')
time_slice2 = slice('2035-01-01T00:00:00.000000000','2055-12-31T23:30:00.000000000')
yearsum_ssp1  = yearsum_ssp.sel(year=time_slice1).mean('year')
yearsum_ssp2  = yearsum_ssp.sel(year=time_slice2).mean('year')
chill_hours_year_ssp1  = chill_hours_year_ssp.sel(year=time_slice1).mean('year')
chill_hours_year_ssp2  = chill_hours_year_ssp.sel(year=time_slice2).mean('year')

#Chilling Hours Model
fig, ax = plt.subplots(figsize=(12, 7))
chill_hours_year.plot(ax = ax, x='year',color = 'cornflowerblue', label='Historical model')
chill_hours_year_ssp.plot(ax = ax, x='year', color = 'midnightblue', label='SSP585 model')
plt.title("Winter chill hours - yearly count for the MPI-ESM1.2-HAM model")
ax.set_ylabel('Hours')
ax.set_xlabel('Years')
ax.axhline(y=chill_hours_year.mean('year'), color='lightsteelblue', label='Historical mean = 2799', xmin=0, xmax=0.46)
ax.axhline(chill_hours_year_ssp1, color='black', label='SSP585 mean 2015-2035 = 2657', xmin=0.46, xmax=0.72)
ax.axhline(chill_hours_year_ssp2, color='black', label='SSP585 mean 2035-2055 = 2342', xmin=0.72, xmax=1)
ax.axhline(2683, color='peachpuff', label='ERA5 historical mean = 2683', linewidth=3)
ax.legend(loc='upper right')
plt.show()

#Utah Model
fig, ax = plt.subplots(figsize=(12, 7))
yearsum.plot(ax = ax, x='year',color = 'cornflowerblue', label='Historical model')
yearsum_ssp.plot(ax = ax, x='year', color = 'midnightblue', label='SSP585 model')
plt.title("Utah chill hour units - yearly count for the MPI-ESM1.2-HAM model")
ax.set_ylabel('Units')
ax.set_xlabel('Years')
ax.axhline(y=yearsum.mean('year'), color='lightsteelblue', label='Historical mean = 2891', xmin=0, xmax=0.46)
ax.axhline(yearsum_ssp1, color='black', label='SSP585 mean 2015-2035 = 3098', xmin=0.46, xmax=0.72)
ax.axhline(yearsum_ssp2, color='black', label='SSP585 mean 2035-2055 =3173', xmin=0.72, xmax=1)
ax.axhline(2563, color='peachpuff', label='ERA5 historical mean chill hours = 2563', linewidth=3)
ax.legend(loc='upper right')
plt.show()

### **Monthly correlation between ERA5 data and bias corrected historical HAMMOZ model:**

In [ ]:
#the opened datasets are interpolated
correra = xr.open_dataset('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//ERA5_2//ERA5.1981-2014_int.nc')
corrmodel = xr.open_dataset('C://Users//frida//OneDrive//Dokument//Klimatdatatillampningar//Project//Hammoz_historical//ham_hist_interpo.nc')
time_slice = slice('1981-01-01T00:00:00.000000000','2014-12-31T23:30:00.000000000')
model_sel_years = corrmodel.sel(time=time_slice)
model_sel_months = model_sel_years.tas.sel(time=model_sel_years.time.dt.month.isin([1, 2, 3, 4, 10, 11, 12]))
era_sel_years = correra.sel(time=time_slice)
era_sel_months = era_sel_years.t2m.sel(time=era_sel_years.time.dt.month.isin([1, 2, 3, 4, 10, 11, 12]))

#mean of every month selected
era_monthly_mean = era_sel_months.groupby('time.month').mean(dim='time')
model_monthly_mean = model_sel_months.groupby('time.month').mean(dim='time')

#the correlation
r = sc.pearsonr(era_monthly_mean, model_monthly_mean)
r